# Cython
Cython je výkonný "programovací jazyk" (metajazyk?), který kombinuje jednoduchost Pythonu s rychlostí a efektivitou C. Umožňuje psát kód v Python-like formátu, který je automaticky převeden do C a zkompilován, což významně zefektivňuje danný kód. 

Jelikož cython vyžaduje kompilaci C a C++ kódu, je nutné mít kompilátor. V případě Windows je nutné mít nějakou Microsoft Microsoft Visual C/C++ (MSVC) - [https://wiki.python.org/moin/WindowsCompilers](https://wiki.python.org/moin/WindowsCompilers).

In [ ]:
#!pip install cython

## Základní použití
Pokud chceme využít Cython musíme vyrobit:
- soubor s příponou .pyx ve kterém je zdrojový kód pro Cython
- soubor s informacemi pro kompilátor (setup.py)

### Soubor s příponou .pyx

In [ ]:
%%writefile hello_cython.pyx
print("Ahoj, Cythone!")

### Soubor s informacemi pro kompilátor (setup.py)

In [ ]:
%%writefile setup.py
from setuptools import setup
from Cython.Build import cythonize

setup(
    ext_modules=cythonize("hello_cython.pyx")
)


setup.py je v Pythonu zavedený typ souboru pro kompilaci/sestavování/build/instalaci Python modulů. V tomto souboru je nutné specifikovat, jaký modul se má kompilovat, jaké jsou jeho závislosti a další informace.

V našem případě má soubor tyto řádky:
- `from setuptools import setup` - importuje funkci setup z knihovny setuptools, která se stará o kompilaci
- `from Cython.Build import cythonize` - importuje funkci cythonize z knihovny Cython, která se stará o překlad z .pyx
- `setup(ext_modules=cythonize("cython_test.pyx"))` - volá funkci setup, která se stará o kompilaci a v ní funkci cythonize která dělá překlad z .pyx

In [ ]:
!python setup.py build_ext --inplace

A máme sestaveno!

Vypsalo to mnoho textu, pro nás je důležité volání (v Linuxu gcc) kompilace a její úspěšné dokončení.

Pokud nás zajímá co vlastně příkaz dělá tak toto reprezentují jeho části:
- `python setup.py` - spustí soubor setup.py v Pythonu
- `build_ext` - příkaz pro kompilaci pythonovské extension - tedy něco co půjde naimportovat do Pythonu
- `--inplace` - příkaz pro kompilaci v místě, tedy po kompilaci se výsledek nakopíruje zase sem, jinak by zůstal v adresáři build

Vyzkoušíme tento zkompilovaný modul v Pythonu:

In [ ]:
import hello_cython

To bylo vše, a fungovalo to.

## Cython proměnné, funkce a Numpy
Zatím jsme neudělali moc užitečného. Pojďme se podívat jak se v cythonu vlastně píše kód.

### Deklarace proměnných s konkrétními typy:
- `cdef int a` - deklarace proměnné a typu int
- `cdef float b` - deklarace proměnné a typu float
- `cdef double c` - deklarace proměnné a typu double
- `cdef str d` - deklarace proměnné a typu str

### Deklarace funkcí s konkrétními argumenty a návratovými typy:
- `cdef int add(int a, int b):` - deklarace funkce add, která má 2 argumenty typu int a vrací int takováto funkce **nelze volat z Pythonu**
- `cpdef int add(int a, int b):` - deklarace funkce add, která má 2 argumenty typu int a vrací int takováto funkce **lze volat z Pythonu**

### Deklarace proměnných s typy z Numpy:
Je třeba mít importy jak pro cython tak pro python 
```python
import numpy as np
cimport numpy as cnp
```
Pak se proměnné deklarují jako:
- `cdef cnp.ndarray[cnp.float64_t, ndim=1] a` - deklarace proměnné a typu numpy array s jedním rozměrem a typem float64

Pro rychlou práci s Numpy poli lze funkci nastavit s:
- `@cython.boundscheck(False)` - vypne kontrolu indexů - tedy pokud šáhneme mimo pole tak to nevyhodí chybu - **používat až když víme, že náš kód je správný** - jinak to může vést od tahání hodnot z jiných polí na blízkých paměťových adresách až po segfault
- `@cython.wraparound(False)` - vypne wraparound - tedy vypne možnost použít záporné indexy

## Vyzkoušejme si to na nějakém příkladu

In [ ]:
%%writefile my_dot.pyx
import numpy as np
cimport numpy as np
from libc.math cimport sqrt
import cython # je zde kvůli dekorátorům

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef double my_dot(np.ndarray[np.float64_t, ndim=1] a, np.ndarray[np.float64_t, ndim=1] b):
    cdef int i
    cdef int n = a.shape[0]
    cdef double result = 0.0
    for i in range(n):
        result += a[i] * b[i]
    return result

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef double my_norm(np.ndarray[np.float64_t, ndim=1] a):
    cdef int i
    cdef int n = a.shape[0]
    cdef double result = 0.0
    for i in range(n):
        result += a[i] * a[i]
    return sqrt(result)

* `cimport` je analogie `import`, pracuje ale s Cython definicemi funkcí (.pxd soubory).
* Podobně `libc` je speciální modul Cythonu. Obsahuje funkce z C standardní knihovny. Např. odmocnina je v něm implementována jako `libc.math.sqrt`.

In [ ]:
%%writefile setup.py
from setuptools import setup, Extension
from Cython.Build import cythonize
import numpy as np

ext_modules = [
    Extension(
        name = "my_dot",
        sources=["my_dot.pyx"],
        include_dirs=[np.get_include()],  # include path for NumPy headers
        extra_compile_args=["-O3"],  # optional, to enable optimization flags
    )
]

setup(
    ext_modules=cythonize(ext_modules),
)




Pro větší kódy (používající nějaké knihovny) musíme použít Extension. Extension je třída, která se stará o kompilaci a překlad z cythonu do C. A umožňuje oproti předchozímu příkladu specifikovat:
- jméno modulu (name)
- zdrojové soubory (sources) - v našem případě jen jeden .pyx ale může být více
- závislosti (include_dirs) - v našem případě numpy - nalinkuje se pro kompilaci
- další parametry kompilátoru (extra_compile_args) - v našem případě -O3 - optimizace na úrovni 3

In [ ]:
!python setup.py build_ext --inplace

In [ ]:
import my_dot

In [ ]:
import numpy as np
a = np.random.rand(100_000_000)
b = np.random.rand(100_000_000)


In [ ]:
%timeit c = my_dot.my_dot(a, b)

In [ ]:
%timeit c = np.dot(a, b)

In [ ]:
print(my_dot.my_dot(a, b), np.dot(a, b))

In [ ]:
%timeit c = my_dot.my_norm(a)

In [ ]:
%timeit c = np.linalg.norm(a)

## Cython v Jupyteru
To, že musíme vytvářet .pyx soubor a setup.py je dost nepříjemné pokud chceme okoušet jednoduchou utilitu. 

V Jupyteru/IPythonu je možné použít magic příkaz %%cython, který umožňuje psát cython kód přímo v bunňce Jupyteru.

%%cython přijímá mnoho dodatečných parametrů (to co se objevuje v setup.py), viz dokumentace.

In [ ]:
# je třeba nejdříve načíst Ipython extension pro cython
%load_ext Cython

In [ ]:
%%cython 

import numpy as np
cimport numpy as np
from libc.math cimport sqrt
import cython # je zde kvůli dekorátorům

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef double my_dot2(np.ndarray[np.float64_t, ndim=1] a, np.ndarray[np.float64_t, ndim=1] b):
    cdef int i
    cdef int n = a.shape[0]
    cdef double result = 0.0
    for i in range(n):
        result += a[i] * b[i]
    return result

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef double my_norm2(np.ndarray[np.float64_t, ndim=1] a):
    cdef int i
    cdef int n = a.shape[0]
    cdef double result = 0.0
    for i in range(n):
        result += a[i] * a[i]
    return sqrt(result)

In [ ]:
%timeit c = my_dot2(a, b)

In [ ]:
%timeit c = my_norm2(a)

### Cython toho nabízí mnoho
Podívejte se na http://docs.cython.org co všechno Cython nabízí -- není toho málo, např.

* použití C++
* šablony (templates)
* OpenMP (k tomu se možná ještě dostaneme)
* vytváření C-API
* třídy (cdef classes)